In [1]:
import sys
import os

sys.path.append("src")

from config import DATASET_PATH, TEST_SIZE
from data_loader import load_and_split_dataset
from sift_bow_extractor import (
    extract_color_sift_features,
    create_visual_vocabulary,
    extract_bow_features
)
from classifier import train_and_evaluate_svm, train_and_evaluate_knn
from utils import evaluate_model
from joblib import dump

In [2]:
(train_images, train_labels), (test_images, test_labels), classes = load_and_split_dataset(
    DATASET_PATH, test_size=TEST_SIZE, sample_ratio=1
)

print(f"✅ Loaded: {len(train_images)} training images, {len(test_images)} testing images")

✅ Loaded: 9600 training images, 2400 testing images


In [3]:
train_descriptors = extract_color_sift_features(train_images)
test_descriptors = extract_color_sift_features(test_images)

Extracting RGB SIFT: 100%|██████████████████| 2400/2400 [01:02<00:00, 38.15it/s]


In [4]:
kmeans = create_visual_vocabulary(train_descriptors)

/Users/chenyangyang/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [6]:
X_train = extract_bow_features(train_descriptors, kmeans)
X_test  = extract_bow_features(test_descriptors, kmeans)

In [7]:
svm_model, y_pred_svm = train_and_evaluate_svm(X_train, X_test, train_labels, test_labels)
evaluate_model(test_labels, y_pred_svm, classes)

🔧 Training SVM classifier...
✅ SVM classifier training complete!

═══ Evaluation Summary ═══
✅ Accuracy       : 0.4796
🎯 Precision      : 0.4879
🔁 Recall         : 0.4796
📊 F1-Score       : 0.4483

🧩 Confusion Matrix:
[[ 74   3   3  17   9   1   2   8   5   4   0   0   9  21   4]
 [  4  59   2  30   1   2   0   7   1   6   0  21  16  11   0]
 [  5   1  63  19   3   3   5   5   7  15   1   5   5  21   2]
 [  0  10   0 133   0   0   0   0   0   0   0   7   3   7   0]
 [  4   0   2   9  53  11  16   3   1  30   3   1   8  17   2]
 [  1   0   0   8   3 134   6   0   0   4   0   0   0   4   0]
 [ 10   0   5   3   6  45  53   4   3   4   3   2   5  17   0]
 [  8  11   1  10   0   3   0  46   2   2   0   2  55  20   0]
 [  6   7  10  27   5  12   6  11  12  30   0   3   1  30   0]
 [  1   1   3  10   2   1   0   0   1 128   0   1   4   7   1]
 [  1   6   1   4   0   1   1   2   2   0 111   3   3  25   0]
 [  2  13   2  71   0   0   0   0   2   5   2  56   7   0   0]
 [  0   8   0  32   0   0 

In [8]:
knn_model, y_pred_knn = train_and_evaluate_knn(X_train, X_test, train_labels, test_labels, k=5)
evaluate_model(test_labels, y_pred_knn, classes)

🔧 Training KNN classifier (k=5)...
✅ KNN classifier training complete!

═══ Evaluation Summary ═══
✅ Accuracy       : 0.5958
🎯 Precision      : 0.6140
🔁 Recall         : 0.5958
📊 F1-Score       : 0.5863

🧩 Confusion Matrix:
[[ 93   8   0   7   3   1   1  10   4   1   1   1   7  13  10]
 [  2  97   0  14   3   1   0   8   1   1   1   2  17   7   6]
 [ 14  10  59  13   1   1   2   9  16   4   0   5   5   8  13]
 [  1   5   0 140   0   0   0   1   0   0   0   4   5   4   0]
 [ 14   1   6   4  85   6   7   3   9  15   4   0   3   1   2]
 [  2   0   0   0   1 144   6   0   2   0   0   0   0   3   2]
 [ 13   4   9   1  22  15  66   7  10   1   3   0   1   4   4]
 [  3  11   1   9   0   0   0  91   1   1   1   1  31   9   1]
 [ 13  13   8   6   6   4   1  10  48   9   1   4   4  13  20]
 [  5   2   2   5   2   0   0   0   7 127   0   0   4   1   5]
 [  1   6   0   6   1   0   1   8   2   0 123   2   3   6   1]
 [  2  24   0  45   0   0   0   2   3   3   0  61  15   2   3]
 [  0  10   0  11   

In [9]:
os.makedirs("models", exist_ok=True)

dump(svm_model, "models/svm_model.joblib")
dump(knn_model, "models/knn_model.joblib")

print("✅ Models saved to 'models/' folder.")

✅ Models saved to 'models/' folder.
